Job 1: Mapper

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import json
import nltk
# Load your JSON data
df = pd.read_json('legal_passages.json')



In [2]:
stopwords_file = open("stopwords.txt", "r", encoding="utf-8")
stopwords_list = []

for line in stopwords_file:
    stopwords_list.append(line.strip())

stopwords_file.close()

In [3]:
doc_dict = {}  # Initialize an empty dictionary

for i, row in enumerate(df.id):
    doc_dict[i] = row
print(doc_dict)

{0: 'Luật Viên chức 2010', 1: 'Bộ Luật Dân sự 2015', 2: 'Luật Giáo dục 2019', 3: 'Luật Tổ chức viện kiểm sát nhân dân 2014', 4: 'Luật Dầu khí 2022', 5: 'Luật Thanh tra 2022', 6: 'Luật Phòng, chống bạo lực gia đình 2022', 7: 'Luật Phòng, chống ma túy 2021', 8: 'Luật Cư trú 2020', 9: 'Luật Tiếp cận thông tin 2016', 10: 'Luật An ninh mạng 2018', 11: 'Hiến pháp 2013', 12: 'Luật Du lịch 2017', 13: 'Luật Tố tụng hành chính 2015', 14: 'Luật Hôn nhân và gia đình 2014', 15: 'Luật Điện ảnh 2022', 16: 'Luật Trọng tài thương mại 2010', 17: 'Luật Thanh niên 2020'}


In [4]:
from underthesea import word_tokenize
import re
job_1 = {}
for i, row in enumerate(df.articles):
    for text_data in row:
        text = text_data["text"]
        
        tokens = word_tokenize(text)
        
        for token in tokens:
            token = re.sub(r'[_!@#]', '', token).lower().strip()
#            Check if the token is a number (digits only) or a word
            
            if not re.search(r'[^\w\s\d]+', token):
                if not token in stopwords_list:
                    key = (token, doc_dict[i])
                    
                    if job_1.get(key):
                        job_1[key] += 1
                    else:
                        job_1[key] = 1  
print(job_1)

ModuleNotFoundError: No module named 'underthesea'

In [ ]:
pip install underthesea


In [ ]:
job_2_mapper = {}
for term, doc_id in job_1:
    input_key = (term, doc_id)
    output_key = (doc_id, job_1[input_key], 1)
    if job_2_mapper.get(term):
        job_2_mapper[term].append(output_key)
    else:
        job_2_mapper[term] = [output_key]


In [ ]:
job_2_reducer = {}
for term in job_2_mapper:
    count = 0
    for article in job_2_mapper[term]:
        count += article[2]
    if count <= 16 and count >= 2:
        job_2_reducer[term] = len(doc_dict) / count 
print(job_2_reducer)

In [ ]:
job_3 = {}
for term, doc_id in job_1:
    input_key = (term, doc_id) 
    job_3[input_key] = job_2_reducer[term] * job_1[input_key]
print(job_3)